In [1]:
%env CUDA_VISIBLE_DEVICES=-1

env: CUDA_VISIBLE_DEVICES=-1


In [2]:
from keras.models import Sequential
from keras.layers import InputLayer, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, RMSprop, Adagrad, Adam
from tensorflow import device

Using TensorFlow backend.


In [3]:
from numpy import load, stack
from itertools import cycle

def bachelor(file, base, batch=32):
    """Generate a batch of training pair of data from directory"""
    with device('/cpu:0'):
        names = open(file+".lst")
        datas = load(file+".npy")
        X = []; y = []; c = 0;
        
        while True:
            for name, data in cycle(zip(names, datas)):
                spec = load(f"{base}/{name.strip()}.npy")
                spec = spec.reshape(spec.shape + (1,))
                para = data.reshape((data.shape[0],))
                X.append(spec); y.append(para); c += 1
                if c == batch:
                    yield stack(X), stack(y)
                    X = []; y = []; c = 0;

## Model

In [4]:
NAME = "lamost"

In [9]:
def new_model(name=NAME):
    """Model definition"""
    model = Sequential(name=name)
#    model.add(InputLayer((64,64,1), name="Input0"))

    model.add(Conv2D(8, (7,7), name="Conv0", input_shape=(64,64,1)))
    model.add(LeakyReLU(name="LeReLU0"))
    model.add(MaxPooling2D((2,2), name="Max0"))
    
    model.add(Conv2D(16, (5,5), name="Conv1"))
    model.add(LeakyReLU(name="LeReLU1"))
    model.add(MaxPooling2D((2,2), padding="same", name="Max1"))
    
    model.add(Conv2D(32, (3,3), name="Conv2"))
    model.add(LeakyReLU(name="LeReLU2"))
    model.add(MaxPooling2D((2,2), padding="same", name="Max2"))
    
    model.add(Flatten(name="Flat"))
    
    model.add(Dropout(0.2, name="Dropout"))    
    
    model.add(Dense(512, activation="tanh"))
    model.add(Dense(128, activation="tanh"))
    model.add(Dense(32, activation="tanh"))
    model.add(Dense(3, activation="linear"))
    return model

In [10]:
MODEL = new_model(NAME)
MODEL.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv0 (Conv2D)               (None, 58, 58, 8)         400       
_________________________________________________________________
LeReLU0 (LeakyReLU)          (None, 58, 58, 8)         0         
_________________________________________________________________
Max0 (MaxPooling2D)          (None, 29, 29, 8)         0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 25, 25, 16)        3216      
_________________________________________________________________
LeReLU1 (LeakyReLU)          (None, 25, 25, 16)        0         
_________________________________________________________________
Max1 (MaxPooling2D)          (None, 13, 13, 16)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 11, 11, 32)        4640      
__________

In [11]:
MODEL.compile(optimizer="sgd", loss="mse", metrics=['accuracy'])

In [12]:
with device('/cpu:0'):
    MODEL.fit_generator(bachelor("train", "reshape"),
                        steps_per_epoch=845, epochs=1,
                        validation_data=bachelor("tests", "reshape"),
                        validation_steps=282)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
845/845 [==============================] - 31s 37ms/step - loss: 0.1624 - acc: 0.8731 - val_loss: 0.1009 - val_acc: 0.9176


In [14]:
MODEL.save(NAME+".h5")